

---
#Problem Statement:
Combine all the seasons data from 2010 to 2025 and predict the remaining results of tthis season including the final league title winner


**Steps**
1.   Combine all 16 seasons of historical data (Premier League 2010-2024 and
Championship 2024-2025).
2.   Train a "master model" on this complete history.

3. Load the current 2025-2026 season's fixtures.

4. Simulate every remaining match of the season using the master model.

5. Tally the results to produce the final predicted Premier League table for the 2025-2026 season.


In [5]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
import warnings
from itertools import permutations

# Ignore potential warnings for a cleaner output
warnings.filterwarnings('ignore')

def standardize_team_name(name):
    """Cleans and standardizes team names to handle inconsistencies."""
    if not isinstance(name, str):
        return name
    name = name.strip()
    mapping = {
        'Man United': 'Man Utd', 'Manchester United': 'Man Utd',
        'Man City': 'Man City', 'Manchester City': 'Man City',
        'Tottenham': 'Spurs',
        'Nott\'m Forest': 'Nottingham Forest',
        'West Brom': 'West Bromwich Albion',
        'QPR': 'Queens Park Rangers',
        'Brighton': 'Brighton & Hove Albion',
        'Wolves': 'Wolverhampton Wanderers'
    }
    return mapping.get(name, name)

# --- 1. Load All Historical Data (2010-2025) ---
historical_files = [f"/content/data/E{year}.csv" for year in range(2010, 2025)] + ["/content/data/E1.csv"]
all_dfs = []
for f in historical_files:
    try:
        df = pd.read_csv(f, encoding='latin1', on_bad_lines='skip')
        all_dfs.append(df)
    except FileNotFoundError:
        print(f"Warning: {f} not found. Skipping.")

if not all_dfs:
    print("FATAL ERROR: No historical data files found. Cannot proceed.")
else:
    historical_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Successfully loaded {len(all_dfs)} historical data files.")

    # --- 2. Clean Data & Engineer Features for the Master Training Set ---
    for col in ['HomeTeam', 'AwayTeam']:
        historical_df[col] = historical_df[col].apply(standardize_team_name)

    relevant_cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    training_data = historical_df[relevant_cols].copy()
    training_data['Date'] = pd.to_datetime(training_data['Date'], dayfirst=True, errors='coerce')
    training_data.dropna(subset=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], inplace=True)
    training_data = training_data.sort_values(by='Date').reset_index(drop=True)

    def get_form_stats(team, date, hist_df):
        """Calculates a team's average goals scored and conceded in their last 5 games."""
        team_df = hist_df[(hist_df['HomeTeam'] == team) | (hist_df['AwayTeam'] == team)]
        past_games = team_df[team_df['Date'] < date].tail(5)
        if len(past_games) < 5: return 1.0, 1.0
        goals_scored = past_games.apply(lambda r: r['FTHG'] if r['HomeTeam'] == team else r['FTAG'], axis=1).mean()
        goals_conceded = past_games.apply(lambda r: r['FTAG'] if r['HomeTeam'] == team else r['FTHG'], axis=1).mean()
        return goals_scored, goals_conceded

    form_features = [get_form_stats(r['HomeTeam'], r['Date'], training_data) + get_form_stats(r['AwayTeam'], r['Date'], training_data) for _, r in training_data.iterrows()]
    training_data = pd.concat([training_data, pd.DataFrame(form_features, columns=['H_GS', 'H_GC', 'A_GS', 'A_GC'])], axis=1).dropna()

    # --- 3. Train the Master Prediction Model ---
    feature_cols = ['HomeTeam', 'AwayTeam', 'H_GS', 'H_GC', 'A_GS', 'A_GC']
    X_train_raw = training_data[feature_cols]
    y_train = training_data['FTR']
    X_train = pd.get_dummies(X_train_raw, columns=['HomeTeam', 'AwayTeam'])

    master_model = LogisticRegression(max_iter=1000, random_state=42)
    master_model.fit(X_train, y_train)
    print("Master model has been trained on all available historical data.")

    # --- 4. Generate the Full 2025-2026 Season Fixture List ---
    try:
        fixtures_25_26_orig = pd.read_csv("/content/data/E2025.csv", encoding='latin1', on_bad_lines='skip')
        for col in ['HomeTeam', 'AwayTeam']:
            fixtures_25_26_orig[col] = fixtures_25_26_orig[col].apply(standardize_team_name)
        fixtures_25_26_orig['Date'] = pd.to_datetime(fixtures_25_26_orig['Date'], dayfirst=True, errors='coerce')

        teams_25_26 = pd.concat([fixtures_25_26_orig['HomeTeam'], fixtures_25_26_orig['AwayTeam']]).unique()

        if len(teams_25_26) < 20:
             print("\nWarning: Fewer than 20 teams found in E2025.csv. The generated fixture list may be incomplete.")

        # Generate all possible home/away combinations
        all_possible_fixtures = list(permutations(teams_25_26, 2))

        # Create a DataFrame of all 380 possible matches
        full_fixture_list = pd.DataFrame(all_possible_fixtures, columns=['HomeTeam', 'AwayTeam'])

        # Merge with the original file to keep the matches that have already been played
        # We use a left merge to keep all generated fixtures, and an indicator to see which ones already exist
        season_fixtures = pd.merge(full_fixture_list, fixtures_25_26_orig, on=['HomeTeam', 'AwayTeam'], how='left', indicator=True)

        # Fixtures that haven't been played yet need a placeholder date to allow the simulation to proceed
        last_known_date = season_fixtures['Date'].max()
        if pd.isna(last_known_date): last_known_date = pd.to_datetime('today') # Fallback if no dates exist

        # Assign incremental dates to unplayed fixtures
        unplayed_mask = season_fixtures['_merge'] == 'left_only'
        num_unplayed = unplayed_mask.sum()
        placeholder_dates = pd.to_datetime([last_known_date + pd.Timedelta(days=i*7) for i in range(1, (num_unplayed // 10) + 2) for _ in range(10)])[:num_unplayed]
        season_fixtures.loc[unplayed_mask, 'Date'] = placeholder_dates

        season_fixtures.sort_values(by='Date', inplace=True)
        season_fixtures.drop(columns=['_merge'], inplace=True)

        print(f"Generated a full fixture list of {len(season_fixtures)} matches for the 2025-2026 season.")

        # --- 5. Simulate the Full Season ---
        live_history = training_data.copy()
        league_table = {team: {'P': 0, 'W': 0, 'D': 0, 'L': 0, 'GF': 0, 'GA': 0, 'Pts': 0} for team in teams_25_26}

        for _, fixture in season_fixtures.iterrows():
            h_team, a_team, date = fixture['HomeTeam'], fixture['AwayTeam'], fixture['Date']

            # Use actual result if it's in the original file
            if pd.notna(fixture['FTR']):
                res, hg, ag = fixture['FTR'], int(fixture['FTHG']), int(fixture['FTAG'])
            # Predict if result is missing
            else:
                h_gs, h_gc = get_form_stats(h_team, date, live_history)
                a_gs, a_gc = get_form_stats(a_team, date, live_history)

                match_data = pd.DataFrame([[h_team, a_team, h_gs, h_gc, a_gs, a_gc]], columns=feature_cols)
                match_data_encoded = pd.get_dummies(match_data, columns=['HomeTeam', 'AwayTeam'])
                match_data_aligned = match_data_encoded.reindex(columns=X_train.columns, fill_value=0)

                res = master_model.predict(match_data_aligned)[0]

                # Estimate goals for the table
                hg = int(round((h_gs + a_gc) / 2)); ag = int(round((a_gs + h_gc) / 2))
                if res == 'H': hg = max(hg, ag + 1)
                elif res == 'A': ag = max(ag, hg + 1)
                else: hg = ag = max(hg, ag)

            # Update league table
            for team, gf, ga, r in [(h_team, hg, ag, res), (a_team, ag, hg, res)]:
                if team in league_table:
                    league_table[team]['P'] += 1; league_table[team]['GF'] += gf; league_table[team]['GA'] += ga
                    if (team == h_team and r == 'H') or (team == a_team and r == 'A'):
                        league_table[team]['W'] += 1; league_table[team]['Pts'] += 3
                    elif r == 'D':
                        league_table[team]['D'] += 1; league_table[team]['Pts'] += 1
                    else:
                        league_table[team]['L'] += 1

            # Add result to history to influence future predictions
            new_row = pd.DataFrame([{'Date': date, 'HomeTeam': h_team, 'AwayTeam': a_team, 'FTHG': hg, 'FTAG': ag, 'FTR': res}])
            live_history = pd.concat([live_history, new_row], ignore_index=True)

        # --- 6. Display Final Predicted Table ---
        final_table = pd.DataFrame.from_dict(league_table, orient='index')
        final_table['GD'] = final_table['GF'] - final_table['GA']
        final_table = final_table.sort_values(by=['Pts', 'GD', 'GF'], ascending=False).reset_index().rename(columns={'index': 'Team'})
        final_table.index += 1

        print("\n--- PREDICTED FINAL PREMIER LEAGUE TABLE 2025-2026 ---")
        print(final_table.to_string())

    except FileNotFoundError:
        print("\nFATAL ERROR: The 2025-2026 season file (E2025.csv) was not found.")

Successfully loaded 16 historical data files.
Master model has been trained on all available historical data.
Generated a full fixture list of 380 matches for the 2025-2026 season.

--- PREDICTED FINAL PREMIER LEAGUE TABLE 2025-2026 ---
                       Team   P   W  D   L   GF   GA  Pts  GD
1                  Man City  38  34  0   4  142  105  102  37
2                   Arsenal  38  34  0   4  140  110  102  30
3                 Liverpool  38  33  0   5  109   80   99  29
4                   Chelsea  38  32  1   5  108   81   97  27
5                     Spurs  38  31  1   6  107   80   94  27
6                   Man Utd  38  31  1   6  109   84   94  25
7                     Leeds  38  20  0  18  105  103   60   2
8                 Newcastle  38  17  1  20  100  103   52  -3
9    Brighton & Hove Albion  38  13  8  17   95   99   47  -4
10           Crystal Palace  38  14  2  22  110  118   44  -8
11                  Everton  38  13  4  21  115  124   43  -9
12        Nottingha

In [8]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings

# Ignore potential warnings for a cleaner output
warnings.filterwarnings('ignore')

def standardize_team_name(name):
    """Cleans and standardizes team names to handle inconsistencies."""
    if not isinstance(name, str):
        return name
    name = name.strip()
    mapping = {
        'Man United': 'Man Utd', 'Manchester United': 'Man Utd',
        'Man City': 'Man City', 'Manchester City': 'Man City',
        'Tottenham': 'Spurs',
        'Nott\'m Forest': 'Nottingham Forest',
        'West Brom': 'West Bromwich Albion',
        'QPR': 'Queens Park Rangers',
        'Brighton': 'Brighton & Hove Albion',
        'Wolves': 'Wolverhampton Wanderers'
    }
    return mapping.get(name, name)

# --- 1. Load and Prepare All Historical Data ---
historical_files = [f"/content/data/E{year}.csv" for year in range(2010, 2025)] + ["/content/data/E1.csv"]
all_dfs = []
for f in historical_files:
    try:
        df = pd.read_csv(f, encoding='latin1', on_bad_lines='skip')
        all_dfs.append(df)
    except FileNotFoundError:
        pass

if not all_dfs:
    print("FATAL ERROR: No historical data files found. Cannot proceed.")
else:
    historical_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Successfully loaded {len(all_dfs)} historical data files.")

    # --- 2. Clean Data & Engineer Features ---
    for col in ['HomeTeam', 'AwayTeam']:
        historical_df[col] = historical_df[col].apply(standardize_team_name)

    relevant_cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    full_data = historical_df[relevant_cols].copy()
    full_data['Date'] = pd.to_datetime(full_data['Date'], dayfirst=True, errors='coerce')
    full_data.dropna(subset=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], inplace=True)
    full_data = full_data.sort_values(by='Date').reset_index(drop=True)

    def get_form_stats(team, date, hist_df):
        """Calculates a team's average goals scored and conceded in their last 5 games."""
        team_df = hist_df[(hist_df['HomeTeam'] == team) | (hist_df['AwayTeam'] == team)]
        past_games = team_df[team_df['Date'] < date].tail(5)
        if len(past_games) < 5: return 1.0, 1.0
        goals_scored = past_games.apply(lambda r: r['FTHG'] if r['HomeTeam'] == team else r['FTAG'], axis=1).mean()
        goals_conceded = past_games.apply(lambda r: r['FTAG'] if r['HomeTeam'] == team else r['FTHG'], axis=1).mean()
        return goals_scored, goals_conceded

    form_features = [get_form_stats(r['HomeTeam'], r['Date'], full_data) + get_form_stats(r['AwayTeam'], r['Date'], full_data) for _, r in full_data.iterrows()]
    full_data = pd.concat([full_data, pd.DataFrame(form_features, columns=['H_GS', 'H_GC', 'A_GS', 'A_GC'])], axis=1)
    full_data.dropna(inplace=True)

    # --- 3. Split Data, Train, and Evaluate ---
    feature_cols = ['HomeTeam', 'AwayTeam', 'H_GS', 'H_GC', 'A_GS', 'A_GC']
    X = full_data[feature_cols]
    y = full_data['FTR']

    # One-Hot Encode the team names
    X_encoded = pd.get_dummies(X, columns=['HomeTeam', 'AwayTeam'])

    # **THIS IS THE KEY STEP**: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

    print(f"\nData split into {len(X_train)} training samples and {len(X_test)} testing samples.")

    # Initialize and train the model ONLY on the training data
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    # --- 4. Compare Performance ---
    # Make predictions on both sets
    preds_on_train = model.predict(X_train)
    preds_on_test = model.predict(X_test)

    # Calculate accuracy on both sets
    accuracy_train = accuracy_score(y_train, preds_on_train)
    accuracy_test = accuracy_score(y_test, preds_on_test)

    print("\n--- Overfitting Analysis ---")
    print(f"Accuracy on Training Data (seen data): {accuracy_train:.2%}")
    print(f"Accuracy on Testing Data (unseen data): {accuracy_test:.2%}")

    difference = abs(accuracy_train - accuracy_test) * 100
    print(f"\nDifference: {difference:.2f}%")

    if difference < 5:
        print("\nConclusion: The model is generalizing well. There is no significant sign of overfitting. 👍")
    else:
        print("\nConclusion: The model may be overfitting. The performance on seen data is notably higher than on unseen data. 👎")

Successfully loaded 16 historical data files.

Data split into 5001 training samples and 1251 testing samples.

--- Overfitting Analysis ---
Accuracy on Training Data (seen data): 54.15%
Accuracy on Testing Data (unseen data): 52.20%

Difference: 1.95%

Conclusion: The model is generalizing well. There is no significant sign of overfitting. 👍




---

#Testing Out Different Other Models
We will test 8 different machine learning models, including industry-standard ones like XGBoost and LightGBM.

**Robust Evaluation**: We'll measure each model on four key metrics: Accuracy, Precision, Recall, and F1-Score. This gives a much more complete picture of performance than just accuracy alone.

**Top 3 Showdown**: After evaluating, we'll identify the top 3 performing models. I will then provide you with a final script to generate the predicted 2025-2026 league table for each of those top 3 models so you can compare their final predictions.

In [10]:
import pandas as pd
import numpy as np
import warnings
from itertools import permutations

# Scikit-learn Models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Modern Boosting Libraries (you may need to !pip install xgboost lightgbm)
import xgboost as xgb
import lightgbm as lgb

# Ignore potential warnings for a cleaner output
warnings.filterwarnings('ignore')

def standardize_team_name(name):
    """Cleans and standardizes team names to handle inconsistencies."""
    if not isinstance(name, str): return name
    name = name.strip()
    mapping = {
        'Man United': 'Man Utd', 'Manchester United': 'Man Utd',
        'Man City': 'Man City', 'Manchester City': 'Man City',
        'Tottenham': 'Spurs',
        'Nott\'m Forest': 'Nottingham Forest',
        'West Brom': 'West Bromwich Albion',
        'QPR': 'Queens Park Rangers',
        'Brighton': 'Brighton & Hove Albion',
        'Wolves': 'Wolverhampton Wanderers'
    }
    return mapping.get(name, name)

# --- 1. Load and Prepare All Historical Data ---
historical_files = [f"/content/data/E{year}.csv" for year in range(2010, 2025)] + ["/content/data/E1.csv"]
all_dfs = []
for f in historical_files:
    try:
        df = pd.read_csv(f, encoding='latin1', on_bad_lines='skip')
        all_dfs.append(df)
    except FileNotFoundError:
        pass

if not all_dfs:
    print("FATAL ERROR: No historical data files found. Cannot proceed.")
else:
    historical_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Successfully loaded {len(all_dfs)} historical data files.")

    # --- 2. Clean Data & Engineer Features ---
    for col in ['HomeTeam', 'AwayTeam']:
        historical_df[col] = historical_df[col].apply(standardize_team_name)

    relevant_cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    full_data = historical_df[relevant_cols].copy()
    full_data['Date'] = pd.to_datetime(full_data['Date'], dayfirst=True, errors='coerce')
    full_data.dropna(subset=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], inplace=True)
    full_data = full_data.sort_values(by='Date').reset_index(drop=True)

    def get_form_stats(team, date, hist_df):
        """Calculates a team's average goals scored and conceded in their last 5 games."""
        team_df = hist_df[(hist_df['HomeTeam'] == team) | (hist_df['AwayTeam'] == team)]
        past_games = team_df[team_df['Date'] < date].tail(5)
        if len(past_games) < 5: return 1.0, 1.0
        goals_scored = past_games.apply(lambda r: r['FTHG'] if r['HomeTeam'] == team else r['FTAG'], axis=1).mean()
        goals_conceded = past_games.apply(lambda r: r['FTAG'] if r['HomeTeam'] == team else r['FTHG'], axis=1).mean()
        return goals_scored, goals_conceded

    form_features = [get_form_stats(r['HomeTeam'], r['Date'], full_data) + get_form_stats(r['AwayTeam'], r['Date'], full_data) for _, r in full_data.iterrows()]
    full_data = pd.concat([full_data, pd.DataFrame(form_features, columns=['H_GS', 'H_GC', 'A_GS', 'A_GC'])], axis=1).dropna()

    # --- 3. Prepare Data for Modeling ---
    feature_cols = ['HomeTeam', 'AwayTeam', 'H_GS', 'H_GC', 'A_GS', 'A_GC']
    X = full_data[feature_cols]
    y = full_data['FTR']

    # Map string labels ('H', 'D', 'A') to integers (2, 1, 0) for model compatibility
    label_mapping = {'H': 2, 'D': 1, 'A': 0}
    y = y.map(label_mapping)

    # One-Hot Encode the team names
    X_encoded = pd.get_dummies(X, columns=['HomeTeam', 'AwayTeam'])

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

    # --- 4. Define and Train All Models ---
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "XGBoost": xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
        "LightGBM": lgb.LGBMClassifier(random_state=42),
        "Support Vector Machine": SVC(random_state=42),
        "K-Nearest Neighbors": KNeighborsClassifier(),
        "Gaussian Naive Bayes": GaussianNB()
    }

    results = {}

    print("\nTraining and evaluating models...")
    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds, average='weighted', zero_division=0)
        recall = recall_score(y_test, preds, average='weighted', zero_division=0)
        f1 = f1_score(y_test, preds, average='weighted', zero_division=0)

        results[name] = [accuracy, precision, recall, f1]
        print(f" ✓ {name} evaluated.")

    # --- 5. Display Comparative Results ---
    results_df = pd.DataFrame.from_dict(results, orient='index',
                                        columns=['Accuracy', 'Precision', 'Recall', 'F1-Score'])

    results_df = results_df.sort_values(by="F1-Score", ascending=False)

    print("\n--- MODEL PERFORMANCE COMPARISON ---")
    print(results_df)

Successfully loaded 16 historical data files.

Training and evaluating models...
 ✓ Logistic Regression evaluated.
 ✓ Random Forest evaluated.
 ✓ Gradient Boosting evaluated.
 ✓ XGBoost evaluated.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000470 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 241
[LightGBM] [Info] Number of data points in the train set: 5001, number of used features: 82
[LightGBM] [Info] Start training from score -1.182411
[LightGBM] [Info] Start training from score -1.410787
[LightGBM] [Info] Start training from score -0.799597
 ✓ LightGBM evaluated.
 ✓ Support Vector Machine evaluated.
 ✓ K-Nearest Neighbors evaluated.
 ✓ Gaussian Naive Bayes evaluated.

--- MODEL PERFORMANCE COMPARISON ---
                        Accuracy  Precision    Recall  F1-Score
Gaussian Naive Bayes    0.492406   0.495678

##Top 3 Performing models based on their F1 Score

Gaussian Naive Bayes (F1-Score: 0.477)

Logistic Regression (F1-Score: 0.467)

LightGBM (F1-Score: 0.465)


In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
import numpy as np
import warnings
from itertools import permutations

# Ignore potential warnings for a cleaner output
warnings.filterwarnings('ignore')

def standardize_team_name(name):
    """Cleans to handle inconsistencies"""
    if not isinstance(name, str): return name
    name = name.strip()
    mapping = {
        'Man United': 'Man Utd', 'Manchester United': 'Man Utd',
        'Man City': 'Man City', 'Manchester City': 'Man City',
        'Tottenham': 'Spurs',
        'Nott\'m Forest': 'Nottingham Forest',
        'West Brom': 'West Bromwich Albion',
        'QPR': 'Queens Park Rangers',
        'Brighton': 'Brighton & Hove Albion',
        'Wolves': 'Wolverhampton Wanderers'
    }
    return mapping.get(name, name)

def get_form_stats(team, date, hist_df):
    """Calculates a team's average goals scored and conceded in their last 5 games."""
    team_df = hist_df[(hist_df['HomeTeam'] == team) | (hist_df['AwayTeam'] == team)]
    past_games = team_df[team_df['Date'] < date].tail(5)
    if len(past_games) < 5: return 1.0, 1.0 # Return neutral form if not enough data
    goals_scored = past_games.apply(lambda r: r['FTHG'] if r['HomeTeam'] == team else r['FTAG'], axis=1).mean()
    goals_conceded = past_games.apply(lambda r: r['FTAG'] if r['HomeTeam'] == team else r['FTHG'], axis=1).mean()
    return goals_scored, goals_conceded

# --- Main Simulation Logic ---
# 1. Load and prepare the full historical dataset
historical_files = [f"/content/data/E{year}.csv" for year in range(2010, 2025)] + ["/content/data/E1.csv"]
all_dfs = []
for f in historical_files:
    try:
        df = pd.read_csv(f, encoding='latin1', on_bad_lines='skip')
        all_dfs.append(df)
    except FileNotFoundError: pass
historical_df = pd.concat(all_dfs, ignore_index=True)
for col in ['HomeTeam', 'AwayTeam']:
    historical_df[col] = historical_df[col].apply(standardize_team_name)
relevant_cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
training_data = historical_df[relevant_cols].copy()
training_data['Date'] = pd.to_datetime(training_data['Date'], dayfirst=True, errors='coerce')
training_data.dropna(subset=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], inplace=True)
training_data = training_data.sort_values(by='Date').reset_index(drop=True)
form_features = [get_form_stats(r['HomeTeam'], r['Date'], training_data) + get_form_stats(r['AwayTeam'], r['Date'], training_data) for _, r in training_data.iterrows()]
training_data = pd.concat([training_data, pd.DataFrame(form_features, columns=['H_GS', 'H_GC', 'A_GS', 'A_GC'])], axis=1).dropna()

# 2. Prepare the final training data
feature_cols = ['HomeTeam', 'AwayTeam', 'H_GS', 'H_GC', 'A_GS', 'A_GC']
X_train_raw = training_data[feature_cols]
y_train = training_data['FTR']

# Map string labels to integers for models that require it
label_mapping = {'H': 2, 'D': 1, 'A': 0}
y_train_mapped = y_train.map(label_mapping)
X_train = pd.get_dummies(X_train_raw, columns=['HomeTeam', 'AwayTeam'])

# ---  Top 3 Models from previous train and test ---
# This dictionary has been updated with your top 3 models based on the F1-Score.
top_3_models = {
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "LightGBM": lgb.LGBMClassifier(random_state=42)
}

# 3. Train the top models and run a simulation for each
for model_name, model in top_3_models.items():
    print(f"\n--- Running Simulation for: {model_name} ---")

    # Train the model on the entire historical dataset
    # Naive Bayes doesn't use random_state. Other models need integer labels.
    if "Naive Bayes" in model_name:
         model.fit(X_train, y_train) # Naive Bayes can handle string labels if needed, but we'll be consistent
    else:
         model.fit(X_train, y_train_mapped)

    # Load and generate the full fixture list for 2025-2026
    fixtures_25_26_orig = pd.read_csv("/content/data/E2025.csv", encoding='latin1', on_bad_lines='skip')
    for col in ['HomeTeam', 'AwayTeam']:
        fixtures_25_26_orig[col] = fixtures_25_26_orig[col].apply(standardize_team_name)
    fixtures_25_26_orig['Date'] = pd.to_datetime(fixtures_25_26_orig['Date'], dayfirst=True, errors='coerce')
    teams_25_26 = pd.concat([fixtures_25_26_orig['HomeTeam'], fixtures_25_26_orig['AwayTeam']]).unique()
    all_possible_fixtures = list(permutations(teams_25_26, 2))
    full_fixture_list = pd.DataFrame(all_possible_fixtures, columns=['HomeTeam', 'AwayTeam'])
    season_fixtures = pd.merge(full_fixture_list, fixtures_25_26_orig, on=['HomeTeam', 'AwayTeam'], how='left', indicator=True)
    last_known_date = season_fixtures['Date'].max()
    if pd.isna(last_known_date): last_known_date = pd.to_datetime('today')
    unplayed_mask = season_fixtures['_merge'] == 'left_only'
    num_unplayed = unplayed_mask.sum()
    placeholder_dates = pd.to_datetime([last_known_date + pd.Timedelta(days=i*7) for i in range(1, (num_unplayed // 10) + 2) for _ in range(10)])[:num_unplayed]
    season_fixtures.loc[unplayed_mask, 'Date'] = placeholder_dates
    season_fixtures.sort_values(by='Date', inplace=True)
    season_fixtures.drop(columns=['_merge'], inplace=True)

    # Run the simulation
    live_history = training_data.copy()
    league_table = {team: {'P': 0, 'W': 0, 'D': 0, 'L': 0, 'GF': 0, 'GA': 0, 'Pts': 0} for team in teams_25_26}

    # Create an inverse mapping to convert predictions back to 'H', 'D', 'A'
    inverse_label_mapping = {v: k for k, v in label_mapping.items()}

    for _, fixture in season_fixtures.iterrows():
        h_team, a_team, date = fixture['HomeTeam'], fixture['AwayTeam'], fixture['Date']
        if pd.notna(fixture['FTR']):
            res, hg, ag = fixture['FTR'], int(fixture['FTHG']), int(fixture['FTAG'])
        else:
            h_gs, h_gc = get_form_stats(h_team, date, live_history)
            a_gs, a_gc = get_form_stats(a_team, date, live_history)
            match_data = pd.DataFrame([[h_team, a_team, h_gs, h_gc, a_gs, a_gc]], columns=feature_cols)
            match_data_encoded = pd.get_dummies(match_data, columns=['HomeTeam', 'AwayTeam'])
            match_data_aligned = match_data_encoded.reindex(columns=X_train.columns, fill_value=0)

            # Get the numeric prediction and convert it back to a string label
            numeric_pred = model.predict(match_data_aligned)[0]
            res = inverse_label_mapping.get(numeric_pred, 'D') # Default to Draw if something goes wrong

            hg = int(round((h_gs + a_gc) / 2)); ag = int(round((a_gs + h_gc) / 2))
            if res == 'H': hg = max(hg, ag + 1)
            elif res == 'A': ag = max(ag, hg + 1)
            else: hg = ag = max(hg, ag)

        for team, gf, ga, r in [(h_team, hg, ag, res), (a_team, ag, hg, res)]:
            if team in league_table:
                league_table[team]['P'] += 1; league_table[team]['GF'] += gf; league_table[team]['GA'] += ga
                if (team == h_team and r == 'H') or (team == a_team and r == 'A'):
                    league_table[team]['W'] += 1; league_table[team]['Pts'] += 3
                elif r == 'D':
                    league_table[team]['D'] += 1; league_table[team]['Pts'] += 1
                else: league_table[team]['L'] += 1

        new_row = pd.DataFrame([{'Date': date, 'HomeTeam': h_team, 'AwayTeam': a_team, 'FTHG': hg, 'FTAG': ag, 'FTR': res}])
        live_history = pd.concat([live_history, new_row], ignore_index=True)

    # Display the final table for the current model
    final_table = pd.DataFrame.from_dict(league_table, orient='index')
    final_table['GD'] = final_table['GF'] - final_table['GA']
    final_table = final_table.sort_values(by=['Pts', 'GD', 'GF'], ascending=False).reset_index().rename(columns={'index': 'Team'})
    final_table.index += 1

    print(f"\n--- PREDICTED FINAL TABLE ({model_name}) ---")
    print(final_table.to_string())


--- Running Simulation for: Gaussian Naive Bayes ---

--- PREDICTED FINAL TABLE (Gaussian Naive Bayes) ---
                       Team   P  W   D  L  GF  GA  Pts  GD
1                  Man City  38  1  37  0  78  74   40   4
2                     Spurs  38  1  37  0  76  73   40   3
3                Sunderland  38  1  37  0  74  71   40   3
4                 Liverpool  38  1  37  0  77  75   40   2
5         Nottingham Forest  38  1  37  0  77  75   40   2
6                     Leeds  38  1  37  0  75  74   40   1
7                   Arsenal  38  1  37  0  75  74   40   1
8                    Fulham  38  0  38  0  75  75   38   0
9    Brighton & Hove Albion  38  0  38  0  74  74   38   0
10           Crystal Palace  38  0  38  0  74  74   38   0
11              Aston Villa  38  0  38  0  71  71   38   0
12                  Chelsea  38  0  38  0  71  71   38   0
13                Newcastle  38  0  38  0  68  68   38   0
14                  Everton  38  0  37  1  74  75   37  -1
15     